In [1]:
#梯度消失和梯度爆炸
from tensorflow import keras
fashion_mnist = keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
print(x_train.shape)
x_train,x_valid = x_train[5000:]/255.0,x_train[:5000]/255.0
y_train,y_valid = y_train[5000:],y_train[:5000]

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.Dense(10,activation='softmax')
])

#非饱和激活函数
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(30,kernel_initializer='he_normal'),    #初始化方式
    keras.layers.LeakyReLU(alpha=0.2),                        #PReLU激活函数 
    #keras.layers.Dense(30,activation='selu',kernel_initializer='lecun_normal')
    keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])#回归不用metrics=['accuracy']


#批量归一化
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,activation='softmax')
])

#梯度裁剪
optimizer = keras.optimizers.SGD(clipvalue=1.0)     #clipnorm = 1.0
model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['accuracy'])

history=model.fit(x_train,y_train,epochs=10,validation_data=(x_valid,y_valid))   #validation_split =0.1


(60000, 28, 28)
Epoch 1/10
1719/1719 [==============================] - 2s 1ms/step - loss: 27.6197 - accuracy: 0.0871 - val_loss: 27.5955 - val_accuracy: 0.0814
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 27.6166 - accuracy: 0.0833 - val_loss: 27.5936 - val_accuracy: 0.0880
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 27.6154 - accuracy: 0.0917 - val_loss: 27.5926 - val_accuracy: 0.0988
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 27.6147 - accuracy: 0.0953 - val_loss: 27.5920 - val_accuracy: 0.1050
Epoch 5/10
1719/1719 [==============================] - 2s 996us/step - loss: 27.6143 - accuracy: 0.0979 - val_loss: 27.5917 - val_accuracy: 0.1074
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 27.6139 - accuracy: 0.0983 - val_loss: 27.5914 - val_accuracy: 0.1090
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 27.6137 - accuracy: 0.1001 - v

In [3]:
#利用keras迁移学习
from tensorflow import keras
import numpy as np

fashion_mnist = keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
print(x_train.shape)
x_train,x_valid = x_train[5000:]/255.0,x_train[:5000]/255.0
y_train,y_valid = y_train[5000:],y_train[:5000]
print('输出的种类数：',len(np.unique(y_valid)))

# model_a = keras.models.Sequential()
# model_a.add(keras.layers.Flatten(input_shape=[28,28]))
# model_a.add(keras.layers.Dense(300,activation='relu'))
# model_a.add(keras.layers.Dense(100,activation='relu'))
# model_a.add(keras.layers.Dense(10,activation='softmax'))
# model_a.compile(loss = 'sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
# history=model_a.fit(x_train,y_train,epochs=10,validation_data=(x_valid,y_valid))   #validation_split =0.1
# model_a.save('my_keras_model.h5')

model_a = keras.models.load_model('my_keras_model.h5')
model_a_clone = keras.models.clone_model(model_a)
model_a_clone.set_weights(model_a.get_weights())
model_b_on_a = keras.models.Sequential(model_a_clone.layers[:-1])
model_b_on_a.add(keras.layers.Dense(10,activation='softmax'))

for layer in model_b_on_a.layers[:-1]:           #冻结层
    layer.trainable = False
model_b_on_a.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
history = model_b_on_a.fit(x_train,y_train,epochs=4,validation_data=(x_valid,y_valid))

for layer in model_b_on_a.layers[:-1]:          #解冻层
    layer.trainable = True
optimizer = keras.optimizers.SGD(lr=1e-4)
model_b_on_a.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history = model_b_on_a.fit(x_train,y_train,epochs=16,validation_data=(x_valid,y_valid))

(60000, 28, 28)
输出的种类数： 10
Epoch 1/4
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5504 - accuracy: 0.8418 - val_loss: 0.3963 - val_accuracy: 0.8684
Epoch 2/4
1719/1719 [==============================] - 2s 969us/step - loss: 0.3664 - accuracy: 0.8770 - val_loss: 0.3671 - val_accuracy: 0.8748
Epoch 3/4
1719/1719 [==============================] - 2s 928us/step - loss: 0.3448 - accuracy: 0.8810 - val_loss: 0.3542 - val_accuracy: 0.8762
Epoch 4/4
1719/1719 [==============================] - 2s 929us/step - loss: 0.3357 - accuracy: 0.8820 - val_loss: 0.3504 - val_accuracy: 0.8796
Epoch 1/16
1719/1719 [==============================] - 3s 1ms/step - loss: 0.3301 - accuracy: 0.8837 - val_loss: 0.3475 - val_accuracy: 0.8790
Epoch 2/16
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3293 - accuracy: 0.8833 - val_loss: 0.3473 - val_accuracy: 0.8800
Epoch 3/16
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3287 - accuracy: 0.8839 - val

In [ ]:
# 更快的优化器
optimizer = keras.optimizers.SGD(lr=0.001,momentum=0.9)    #动量优化 
optimizer = keras.optimizers.SGD(lr=0.001,momentum=0.9,nesterov=True)   #Nesterov加速梯度
optimizer = keras.optimizers.RMSprop(lr=0.001,rho=0.9)   #RMSprop
optimizer = keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999)



#学习率调度
optimizer =keras.optimizers.SGD(lr=0.01,decay=1e-4)  #幂调度

def exponential_decay_fn(epoch):
    return 0.01*0.1**(epoch/20)
def exponential_decay(lro,s):
    def exponential_decay_fn(epoch):
        return lro*0.1**(epoch/s)
    return exponential_decay_fn
exponential_decay_fn = exponential_decay(lro=0.01,s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(x_train_scaled,y_train,[...],callbacks=[lr_scheduler])


In [ ]:
#正则化避免过拟合 

#L1和L2 正则化 
#如果想要稀疏模型用L1，许多权重为0；L2用于约束神经网络的连接权重
layer = keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))  #keras.regularizers.l1()   keras.regularizers.l1_l2()  


#Dropout 
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300,activation='elu',kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10,activation='softmax')
])

#MC Dropout
y_probas = np.stack([model(x_test_scaled,training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)

np.round(model.predict(x_test_scaled[:1]),2)  #没有丢弃层
np.round(y_probas[:,:1],2)   #Dropout
np.round(y_proba[:1],2)      #MC Dropout    #适用于构建风险敏感的应用  

#最大范数正则化 
keras.layers.Dense(100,activation='elu',kernel_initializer='he_normal',kernel_constraint=keras.constraints.max_norm(1.0))